In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linear_sum_assignment
from collections import Counter
import itertools
import copy

In [2]:
#100*100 matching market
# 示例数据
donors_names = ['d' + str(i) for i in range(1, 101)]
recipients_names = ['r' + str(i) for i in range(1, 101)]

# 创建初始匹配
initial_match = list(zip(donors_names, recipients_names))

# 加载矩阵数据
don_pref_matrix = pd.read_csv('C:\\Users\\Hongan Li\\Desktop\\Research Project\\Code\\final_code\\preference matrices\\don_pre_100_2.csv', index_col=0)
reci_pref_matrix = pd.read_csv('C:\\Users\\Hongan Li\\Desktop\\Research Project\\Code\\final_code\\preference matrices\\reci_pre_100_2.csv', index_col=0)

In [ ]:
#20*20 matching market

# 示例数据
donors_names = ['d' + str(i) for i in range(1, 21)]
recipients_names = ['r' + str(i) for i in range(1, 21)]

# 创建初始匹配
initial_match = list(zip(donors_names, recipients_names))

# 加载矩阵数据
don_pref_matrix = pd.read_csv('C:\\Users\\Hongan Li\\Desktop\\Research Project\\Code\\final_code\\preference matrices\\don_pre_20_2.csv', index_col=0)
reci_pref_matrix = pd.read_csv('C:\\Users\\Hongan Li\\Desktop\\Research Project\\Code\\final_code\\preference matrices\\reci_pre_20_2.csv', index_col=0)

In [5]:
def find_all_instabilities(matching, reci_pre, don_pre):
    matching_dict = {donor: recipient for donor, recipient in matching}
    instabilities = {}

    for donor, recipient in matching_dict.items():
        # Index of the recipient in the donor's preference list
        matched_recipient_rank = don_pre.loc[donor, recipient]
        donor_preferences = don_pre.loc[donor]

        # Check all recipients above the matched recipient in donor's preferences
        preferred_recipients = donor_preferences[donor_preferences < matched_recipient_rank].index.tolist()

        for other_recipient in preferred_recipients:
            # Find the other donor matched to the other recipient
            other_donor_list = [d for d, r in matching if r == other_recipient]
            if other_donor_list:
                other_donor = other_donor_list[0]

                # If the other recipient prefers this donor over their current match
                if reci_pre.loc[other_recipient, donor] < reci_pre.loc[other_recipient, other_donor]:
                    donor_preference_score = don_pre.loc[donor, other_recipient]
                    recipient_preference_score = reci_pre.loc[other_recipient, donor]
                    instabilities[(donor, other_recipient)] = (donor_preference_score, recipient_preference_score)
    
    return instabilities

In [3]:
## Gale-Shapley Algorithm best to recipients:
def gale_shapley_1(don_pre, reci_pre):
    don_available = {recipient: donors_names[:] for recipient in recipients_names}
    waiting_list = []
    proposals = {}
    
    while len(waiting_list) < len(recipients_names):
        for recipient in recipients_names:
            if recipient not in waiting_list:
                donor = don_available[recipient]
                best_choice = reci_pre.loc[recipient][reci_pre.loc[recipient].index.isin(donor)].idxmin()
                proposals[(recipient, best_choice)] = (reci_pre.loc[recipient][best_choice], don_pre.loc[best_choice][recipient])
        
        overlays = Counter([key[1] for key in proposals.keys()])
        
        for donor in overlays.keys():
            if overlays[donor] > 1:
                pairs_to_drop = sorted({pair: proposals[pair] for pair in proposals.keys() if donor in pair}.items(), key=lambda x: x[1][1])[1:]
                for p_to_drop in pairs_to_drop:
                    del proposals[p_to_drop[0]]
                    _donor = copy.copy(don_available[p_to_drop[0][0]])
                    _donor.remove(p_to_drop[0][1])
                    don_available[p_to_drop[0][0]] = _donor
        
        waiting_list = [recipient[0] for recipient in proposals.keys()]
    
    return proposals

## Gale-Shapley Algorithm reversed:
def gale_shapley_2(reci_pre, don_pre):
    reci_available = {donor: recipients_names[:] for donor in donors_names}
    waiting_list = []
    proposals = {}
    
    while len(waiting_list) < len(donors_names):
        for donor in donors_names:
            if donor not in waiting_list:
                recipient = reci_available[donor]
                best_choice = don_pre.loc[donor][don_pre.loc[donor].index.isin(recipient)].idxmin()
                proposals[(donor, best_choice)] = (don_pre.loc[donor][best_choice], reci_pre.loc[best_choice][donor])
        
        overlays = Counter([key[1] for key in proposals.keys()])
        
        for recipient in overlays.keys():
            if overlays[recipient] > 1:
                pairs_to_drop = sorted({pair: proposals[pair] for pair in proposals.keys() if recipient in pair}.items(), key=lambda x: x[1][1])[1:]
                for p_to_drop in pairs_to_drop:
                    del proposals[p_to_drop[0]]
                    _recipient = copy.copy(reci_available[p_to_drop[0][0]])
                    _recipient.remove(p_to_drop[0][1])
                    reci_available[p_to_drop[0][0]] = _recipient
        
        waiting_list = [donor[0] for donor in proposals.keys()]
    
    return proposals

In [8]:
recipients_names = reci_pref_matrix.index.tolist()
donors_names = don_pref_matrix.index.tolist()

# Find the optimal match for recipients
mu_R = gale_shapley_2(reci_pref_matrix, don_pref_matrix)

print("recipients optimal stable matching:", mu_R)

recipients optimal stable matching: {('d99', 'r30'): (0, 17), ('d56', 'r24'): (0, 5), ('d89', 'r78'): (0, 5), ('d48', 'r93'): (0, 32), ('d40', 'r26'): (0, 17), ('d63', 'r74'): (0, 40), ('d34', 'r60'): (0, 0), ('d52', 'r34'): (0, 10), ('d46', 'r55'): (0, 14), ('d51', 'r19'): (0, 50), ('d42', 'r94'): (0, 47), ('d64', 'r91'): (0, 29), ('d67', 'r28'): (0, 8), ('d77', 'r84'): (0, 23), ('d50', 'r33'): (0, 14), ('d54', 'r16'): (0, 7), ('d41', 'r63'): (0, 4), ('d31', 'r86'): (0, 41), ('d49', 'r38'): (0, 18), ('d72', 'r51'): (0, 10), ('d38', 'r81'): (0, 5), ('d12', 'r98'): (0, 25), ('d88', 'r61'): (0, 14), ('d13', 'r73'): (0, 0), ('d22', 'r59'): (0, 10), ('d10', 'r11'): (0, 78), ('d68', 'r90'): (0, 0), ('d25', 'r100'): (1, 6), ('d8', 'r65'): (1, 12), ('d75', 'r45'): (1, 34), ('d55', 'r9'): (1, 33), ('d65', 'r21'): (1, 60), ('d37', 'r72'): (1, 3), ('d66', 'r32'): (1, 35), ('d18', 'r49'): (2, 8), ('d85', 'r12'): (1, 75), ('d1', 'r25'): (2, 29), ('d74', 'r68'): (1, 17), ('d83', 'r41'): (2, 66), ('

In [9]:
instabilty = find_all_instabilities(mu_R, reci_pref_matrix, don_pref_matrix)

if instabilty:
    print('The blocking pairs are:')
    for i in instabilty:
        print(i)
else:
    print('The matching is stable.')

The matching is stable.
